In [1]:
import yaml
config = "motif_params_1.yaml"
with open(config,'r') as f:
    motif_params = yaml.load(f)

    
print(motif_params)

{'motif_outlier': 'None', 'out_f': None, 'name': 'processed/CHO_GROCap1_GRO1', 'ncomp': 10, 'dim_red': 'svd', 'motif_list': '()', 'peak_outlier': 0, 'peak_z_outlier': True, 'cluster_alg': 'hdb', 'class_alg': 'None', 'group_names': ['Promoter', 'Enhancer'], 'group_files': ['processed/sample_CHO_GROCap1_and_CHO_GRO1.tsv', 'processed/enhancer_CHO_GROCap1_and_CHO_GRO1'], 'norm': 'None'}


In [2]:
## Parameters specific to where your folders are and your data
parameter_file = '../params/params.yaml'

import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
anno_gff=doc['annotation']
mRNA_peak_file = doc["mRNA_peak_file"]
mRNA_peak_150_file = doc["mRNA_peak_150_file"]
supplemental = doc["supplemental"]
tss_annotation = doc['tss_annotation']
import annotation
import os
os.chdir
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
import time

from os.path import basename
from Homer import *
from create_output import *
import motif_analysis
print('Number of tissues: ',len(tissues))

from sklearn.decomposition import TruncatedSVD
from scipy import sparse as sp

('Number of tissues: ', 13)


## Create different groups

## Promoter just copy the file to here

In [3]:
## Promoter
in_f = "../Results/tss_annotation_peaks/sample_CHO_GROCap1_and_CHO_GRO1.tsv" 
out_f = "processed/sample_CHO_GROCap1_and_CHO_GRO1.tsv"
cmd = "cp {in_f} {out_f}".format(in_f=in_f,out_f=out_f)
print(cmd)

!{cmd}

cp ../Results/tss_annotation_peaks/sample_CHO_GROCap1_and_CHO_GRO1.tsv processed/sample_CHO_GROCap1_and_CHO_GRO1.tsv


## Enhancer use filt_peaks

In [4]:
anno_df = pd.read_csv(motif_params["name"] + ".anno" ,sep="\t",index_col=0)
anno_df.head()

,Chr,Start,End,Strand,Normalized Tag Count,focus ratio,findPeaks Score,Total Tags (normalized to Control Experiment),Control Tags,Fold Change vs Control,p-value vs Control,Fold Change vs Local,p-value vs Local,Dispersion Ratio,Periodic Ratio,Nearest TSS,Distance to TSS,isSameStrand
#PeakID,,,,,,,,,,,,,,,,,,
NW_020822465.1-1,NW_020822465.1,12601780,12601930,+,3959.2,0.920,105445.0,82497.3,8891.0,9.28,0.0,9985.41,0.0,0.860768,0.510682,NM_001246742.1,-24672.0,False
NW_020822391.1-1,NW_020822391.1,705573,705723,+,887.1,0.891,23648.0,18485.0,7306.0,2.53,0.0,2692.68,0.0,0.859391,0.392157,XM_027389096.1,-326.0,False
NW_020822519.1-1,NW_020822519.1,30535313,30535463,+,821.9,0.886,22669.0,17125.6,5189.0,3.30,0.0,335.22,0.0,0.919301,0.543109,XM_027411336.1,-18627.0,False
NW_020822618.1-1,NW_020822618.1,4328685,4328835,-,737.3,0.991,19598.0,15363.1,4836.0,3.18,0.0,6598.99,0.0,0.996479,0.275362,XM_027428396.1,21529.0,True
NW_020822465.1-2,NW_020822465.1,12627307,12627457,+,639.4,0.929,17120.0,13324.0,2049.0,6.50,0.0,763.86,0.0,0.896852,0.575585,NM_001246742.1,-50199.0,False


In [5]:
peak_by_motif = pd.read_csv(motif_params["name"] + ".motifCount" ,sep="\t",index_col=0)
peak_by_motif.head()

,ap1.motif,ap2.motif,ap2gamma.motif,ap4.motif,ar-half.motif,are-fox.motif,are.motif,arnt.motif,ascl1.motif,atf1.motif,...,znf467.motif,znf519.motif,znf528.motif,znf652.motif,znf669.motif,znf675.motif,znf692.motif,znf7.motif,znf711.motif,zscan22.motif
#PeakID,,,,,,,,,,,,,,,,,,,,,
NW_020822465.1-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NW_020822391.1-1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
NW_020822519.1-1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NW_020822618.1-1,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,1,0,0,3
NW_020822465.1-2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
%%time

## Enhancer
enh_f = motif_params["group_files"][1]#"processed/enhancer_CHO_GROCap1_and_CHO_GRO1"
enh_df = annotation.filt_peaks(
    anno_df,
    distance=(40000),
    thresh=1,
    thresh_col="Normalized Tag Count",
    divergent_thr=500,
    save_f=enh_f)

CPU times: user 52.5 s, sys: 32 ms, total: 52.5 s
Wall time: 53.1 s


In [7]:
motif_analysis.create_groups(motif_params["group_files"], motif_params["group_names"], "processed/groups")

{'NW_020822470.1-247': 'Enhancer',
 'NW_020822567.1-1907': 'Promoter',
 'NW_020822470.1-248': 'Promoter',
 'NW_020822398.1-251': 'Enhancer',
 'NW_020822393.1-164': 'Promoter',
 'NW_020822435.1-86': 'Enhancer',
 'NW_020822370.1-279': 'Promoter',
 'NW_020822634.1-88': 'Promoter',
 'NW_020822439.1-3004': 'Enhancer',
 'NW_020822487.1-74': 'Promoter',
 'NW_020822487.1-73': 'Promoter',
 'NW_020822487.1-72': 'Promoter',
 'NW_020822487.1-71': 'Promoter',
 'NW_020822634.1-81': 'Promoter',
 'NW_020822570.1-14': 'Promoter',
 'NW_020822439.1-1150': 'Promoter',
 'NW_020822507.1-500': 'Promoter',
 'NW_020822438.1-131': 'Promoter',
 'NW_020822438.1-130': 'Promoter',
 'NW_020822421.1-32': 'Promoter',
 'NW_020822467.1-161': 'Enhancer',
 'NW_020822467.1-166': 'Enhancer',
 'NW_020822421.1-36': 'Promoter',
 'NW_020822421.1-34': 'Promoter',
 'NW_020822458.1-16': 'Enhancer',
 'NW_020822577.1-724': 'Enhancer',
 'NW_020822468.1-68': 'Promoter',
 'NW_020822577.1-723': 'Enhancer',
 'NW_020822577.1-721': 'Enhanc